## Before you start:
**Tools → Settings → Editor → completions / suggestions / linting → disable**

**Task 1:** Study how basic reference counting works
Create a string and investigate its reference count at different stages. Return a tuple: (count_after_creation, count_after_ref, count_after_del)

In [36]:


import sys

def task1():
    s = "hello"
    count_after_creation = sys.getrefcount(s)  # count including temporary reference

    # Add another reference
    ref = s
    count_after_ref = sys.getrefcount(s)

    # Remove the extra reference
    del ref
    count_after_del = sys.getrefcount(s)

    return (count_after_creation, count_after_ref, count_after_del)


# Check
print("Task 1:", task1())




Task 1: (4294967295, 4294967295, 4294967295)


In [ ]:
import sys
import gc
import weakref

**Task 2:** Function impact on Reference Counting
Create a list and pass it to a function.Compare reference count before and after function call.Return a tuple: (count_before_call, count_during_call, count_after_call)

In [44]:
import sys

def task2():

    def process_list(lst):
        # lst receives an extra reference because it's a function argument
        return sys.getrefcount(lst)

    my_list = [1, 2, 3]

    count_before_call = sys.getrefcount(my_list)

    # During call: lst inside the function becomes an additional reference
    count_during_call = process_list(my_list)

    # After the function returns, that temporary reference is gone
    count_after_call = sys.getrefcount(my_list)

    return (count_before_call, count_during_call, count_after_call)


# Check
print("Task 2:", task2())


Task 2: (2, 3, 2)


**Task 3:** Cyclic references and Memory Leaks
Create two objects with a cyclic reference, then break it.Use gc to check the number of collected objects.Return the number of objects collected by gc after breaking the reference.

In [48]:
import gc

def task3():
    class Node:
        def __init__(self, name):
            self.name = name
            self.ref = None

    # Clean up any existing garbage so we only see *our* cycle
    gc.collect()

    # Create two nodes with a cyclic reference
    a = Node("a")
    b = Node("b")
    a.ref = b
    b.ref = a

    # Remove our own references; only the cycle keeps them alive now
    del a
    del b

    # Now GC should detect and collect the unreachable cycle
    collected = gc.collect()
    return collected


print("Task 3:", task3())


Task 3: 2


**Task 4:** Comparing Reference Count for different data types
Compare reference count for numbers, strings, lists and dictionaries.Return a dictionary with reference count for each type after creation.

In [49]:
import sys

def task4():
    num = 42
    text = "hello"
    lst = [1, 2, 3]
    dct = {"a": 1}

    return {
        "number": sys.getrefcount(num),
        "string": sys.getrefcount(text),
        "list": sys.getrefcount(lst),
        "dict": sys.getrefcount(dct)
    }

# Check
print("Task 4:", task4())


Task 4: {'number': 4294967295, 'string': 4294967295, 'list': 2, 'dict': 2}


**Task 5:** Weak References
Create two objects with a weak reference between them.Ensure the objects can be deleted by the garbage collector.Return True if the weakref does not increase reference count.

In [50]:
import sys
import weakref
import gc

def task5():
    class Data:
        def __init__(self, value):
            self.value = value

    # Create object
    obj = Data(10)

    # Reference count before weakref
    count_before = sys.getrefcount(obj)

    # Create weak reference
    w = weakref.ref(obj)

    # Reference count after weakref
    count_after = sys.getrefcount(obj)

    # Remove the strong reference
    del obj

    # Run garbage collector
    gc.collect()

    # Weak reference should now return None (object collected)
    collected = w() is None

    # True if weakref did NOT change reference count AND object was collected
    return (count_before == count_after) and collected


print("Task 5:", task5())


Task 5: True


**Task 6:** Monitoring the Garbage Collector
Register a callback function to track GC activity.Return a list of events tracked by the callback.

In [56]:
import gc

def task6():
    events = []

    # Callback function that records GC events
    def callback(phase, info):
        events.append((phase, info.get("generation", None)))

    # Register the callback
    gc.callbacks.append(callback)

    # Trigger garbage collection manually
    gc.collect()

    # Remove the callback so it does not affect other tasks
    gc.callbacks.remove(callback)

    return events


print("Task 6:", task6())


Task 6: [('start', 2), ('stop', 2)]


**Task 7:** GC Generation Analysis
Create objects and trace their movement between GC generations.Return a tuple with the number of objects in each generation before and after object creation.

In [63]:
import gc

def task7():
    # Start from a clean state
    gc.collect()

    # Number of tracked allocations per generation BEFORE creating new objects
    before = tuple(gc.get_count())  # (gen0, gen1, gen2)

    # Create a bunch of container objects that are tracked by the GC
    objs = []
    for _ in range(10000):
        objs.append([0] * 10)

    # Force collection of younger generations so survivors can be promoted
    gc.collect(0)
    gc.collect(1)

    # Numbers AFTER object creation and some GC activity
    after = tuple(gc.get_count())

    # Return (before, after), each is a 3-tuple for generations 0, 1, 2
    return (before, after)


print("Task 7:", task7())


Task 7: ((12, 0, 0), (0, 0, 2))


**Task 8:** Monitoring Garbage Collection Thresholds
Study how GC generation counters change when creating objects.Return a dictionary with the state of the counters before and after object creation.

In [68]:
import gc

def task8():
    gc.collect()

    initial_threshold = gc.get_threshold()
    initial_count = gc.get_count()

    # Create many GC-tracked objects
    objs = []
    for _ in range(10000):
        objs.append([0] * 10)

    after_creation_count = gc.get_count()

    collected = gc.collect(0)
    after_collect_count = gc.get_count()

    result = {
        'thresholds': initial_threshold,
        'initial_count': initial_count,
        'after_creation': after_creation_count,
        'after_collect_gen0': after_collect_count,
        'collected_objects': collected
    }

    # Print each key/value on a new line
    for key, value in result.items():
        print(f"{key}: {value}")

    return result


print("Task 8:", task8())



thresholds: (700, 10, 10)
initial_count: (1, 0, 0)
after_creation: (190, 2, 1)
after_collect_gen0: (0, 3, 1)
collected_objects: 0
Task 8: {'thresholds': (700, 10, 10), 'initial_count': (1, 0, 0), 'after_creation': (190, 2, 1), 'after_collect_gen0': (0, 3, 1), 'collected_objects': 0}


**Task 9:** Quick GC check
Check if the GC collects cyclic references.Return the difference in the number of objects before and after collection.

In [97]:
import gc

def task9():
    class Node:
        def __init__(self):
            self.ref = None

    # Start from a clean state
    gc.collect()
    before = len(gc.get_objects())

    # Create a cyclic reference that becomes unreachable
    def make_cycle():
        a = Node()
        b = Node()
        a.ref = b
        b.ref = a
        # After this function returns, only the cycle keeps them alive

    make_cycle()

    # Run GC to collect the unreachable cycle
    gc.collect()
    after = len(gc.get_objects())

    # Return the difference in the number of tracked objects
    return before - after


print("Task 9:", task9())


Task 9: 22


In [98]:
def task9_solution():
    # Quickly create circular references
    a, b, c = [], [], []
    a.append(b); b.append(c); c.append(a)  # Cycle a->b->c->a

    count_before = len(gc.get_objects())
    collected = gc.collect()
    count_after = len(gc.get_objects())

    return count_before - count_after, collected

print("Task 9:", task9_solution())

Task 9: (38, 6)


**Task 10:** Detector for "undying" objects
Find objects that survive forced garbage collection.Output the number of such objects.

In [107]:
import gc

def task10():
    # Make sure we start from a relatively clean state
    gc.collect()

    # Snapshot of currently tracked objects
    objs_before = gc.get_objects()
    ids_before = {id(o) for o in objs_before}

    # Force GC again
    gc.collect()

    # Snapshot after forced collection
    objs_after = gc.get_objects()
    ids_after = {id(o) for o in objs_after}

    # "Undying" objects: present both before and after
    undying_ids = ids_before & ids_after

    return len(undying_ids)


print("Task 10:", task10())


Task 10: 114185
